In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [10]:
data=pd.read_csv("../merged/len4_ndb.csv",encoding="shift-jis",index_col=0)
# drop never treated group
data=data.dropna(subset=["Approved"])
data.head()

C:\Users\hakos\AppData\Local\Temp\ipykernel_2848\298005430.py:1: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("../merged/len4_ndb.csv",encoding="shift-jis",index_col=0)


,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,oral,id_l4,Approved,KEGG,Ingredient,variable,id,revenue
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,0,1145,1985.0,D00141,Indometacin,id1,1145.0,5.639285e+06
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,0,1145,1985.0,D00141,Indometacin,id1,1145.0,2.486458e+06
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,0,1145,1985.0,D00141,Indometacin,id1,1145.0,2.052130e+06
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,0,1145,1985.0,D00141,Indometacin,id1,1145.0,1.270436e+07
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,0,1145,1985.0,D00141,Indometacin,id1,1145.0,8.081441e+06


後発区分を廃す

1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [11]:
data["elasped"]=data["year"]-data["Approved"]
#　不要なところを除外
# data.loc[(data['elasped'] > 10) | (data["elasped"]< -5), 'elasped'] = np.nan

# dummy をとりたいところを分離
dummies=pd.get_dummies(data["elasped"],prefix="elasped")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
# sumをとりたいところを分離
sumcolumns=["総計","revenue"]
sums=data.groupby(["year","id"])[sumcolumns].sum().reset_index()
# save temporarily
colnames=dummies.columns.to_list()
# count ver.
# aggregated_data = pd.concat([data.groupby(['year', 'id']).first(), dummies.groupby([data['year'], data['id']]).sum()], axis=1).reset_index()

# dummy ver.
aggregated_df = pd.concat([
    data.groupby(['year', 'id']).first(), 
    dummies.groupby([data['year'], data['id']]).apply(lambda x: (x[colnames] > 0).any().astype(int))
], axis=1).reset_index()
# sumをとったところを追加する
aggregated_df[sumcolumns]=sums[sumcolumns]
aggregated_df
aggregated_df.head()

C:\Users\hakos\AppData\Local\Temp\ipykernel_2848\2258092850.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["elasped"]=data["year"]-data["Approved"]


,year,id,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,...,elasped_29,elasped_30,elasped_31,elasped_32,elasped_33,elasped_34,elasped_35,elasped_36,elasped_37,elasped_38
0,2014,1145.0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,1.604335e+06,...,1,0,0,0,0,0,0,0,0,0
1,2014,1147.0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,1.652349e+08,...,0,0,0,0,0,0,0,0,0,0
2,2014,1149.0,114.0,解熱鎮痛消炎剤,620098801.0,ロキソニン錠６０ｍｇ,1149019F1560,17.5,0.0,2.285302e+10,...,1,0,0,0,0,0,0,0,0,0
3,2014,1219.0,121.0,局所麻酔剤,620005991.0,ストロカイン錠５ｍｇ,1219002F1065,5.7,0.0,4.840017e+07,...,0,0,0,0,0,0,0,0,0,0
4,2014,1231.0,123.0,自律神経剤,620002526.0,チアトンカプセル１０ｍｇ,1231013M2179,15.2,0.0,1.596449e+08,...,0,0,0,0,0,0,0,0,0,0


In [12]:
colnames

['elasped_m8',
 'elasped_m7',
 'elasped_m6',
 'elasped_m5',
 'elasped_m4',
 'elasped_m3',
 'elasped_m2',
 'elasped_m1',
 'elasped_0',
 'elasped_1',
 'elasped_2',
 'elasped_3',
 'elasped_4',
 'elasped_5',
 'elasped_6',
 'elasped_7',
 'elasped_8',
 'elasped_9',
 'elasped_10',
 'elasped_11',
 'elasped_12',
 'elasped_13',
 'elasped_14',
 'elasped_15',
 'elasped_16',
 'elasped_17',
 'elasped_18',
 'elasped_19',
 'elasped_20',
 'elasped_21',
 'elasped_22',
 'elasped_23',
 'elasped_24',
 'elasped_25',
 'elasped_26',
 'elasped_27',
 'elasped_28',
 'elasped_29',
 'elasped_30',
 'elasped_31',
 'elasped_32',
 'elasped_33',
 'elasped_34',
 'elasped_35',
 'elasped_36',
 'elasped_37',
 'elasped_38']

connect generic usage

In [13]:
dusage=pd.read_csv("../generic/generic_usage_imp.csv")
dusage

,Year,Percentage
0,2007,34.90
1,2008,35.35
2,2009,35.80
3,2010,37.85
4,2011,39.90
5,2012,43.40
6,2013,46.90
7,2014,51.55
8,2015,56.20
9,2016,61.00


In [14]:
merged_df=aggregated_df.merge(dusage,left_on="year",right_on="Year")
merged_df.head()

,year,id,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,...,elasped_31,elasped_32,elasped_33,elasped_34,elasped_35,elasped_36,elasped_37,elasped_38,Year,Percentage
0,2014,1145.0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,1.604335e+06,...,0,0,0,0,0,0,0,0,2014,51.55
1,2014,1147.0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,1.652349e+08,...,0,0,0,0,0,0,0,0,2014,51.55
2,2014,1149.0,114.0,解熱鎮痛消炎剤,620098801.0,ロキソニン錠６０ｍｇ,1149019F1560,17.5,0.0,2.285302e+10,...,0,0,0,0,0,0,0,0,2014,51.55
3,2014,1219.0,121.0,局所麻酔剤,620005991.0,ストロカイン錠５ｍｇ,1219002F1065,5.7,0.0,4.840017e+07,...,0,0,0,0,0,0,0,0,2014,51.55
4,2014,1231.0,123.0,自律神経剤,620002526.0,チアトンカプセル１０ｍｇ,1231013M2179,15.2,0.0,1.596449e+08,...,0,0,0,0,0,0,0,0,2014,51.55


In [15]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies.head()

,year,id,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,...,id_3327.0,id_3399.0,id_3919.0,id_3992.0,id_3999.0,id_4413.0,id_4490.0,id_6250.0,id_6290.0,id_7990.0
0,2014,1145.0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,1.604335e+06,...,0,0,0,0,0,0,0,0,0,0
1,2014,1147.0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,1.652349e+08,...,0,0,0,0,0,0,0,0,0,0
2,2014,1149.0,114.0,解熱鎮痛消炎剤,620098801.0,ロキソニン錠６０ｍｇ,1149019F1560,17.5,0.0,2.285302e+10,...,0,0,0,0,0,0,0,0,0,0
3,2014,1219.0,121.0,局所麻酔剤,620005991.0,ストロカイン錠５ｍｇ,1219002F1065,5.7,0.0,4.840017e+07,...,0,0,0,0,0,0,0,0,0,0
4,2014,1231.0,123.0,自律神経剤,620002526.0,チアトンカプセル１０ｍｇ,1231013M2179,15.2,0.0,1.596449e+08,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
df_with_dummies.to_csv("../merged/len4_ndb_agg_all_DropNever.csv",encoding="shift-jis")